In [1]:
from __future__ import division
from IPython.display import display
from IPython.display import Markdown as md
from IPython.core.display import SVG
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.odr import *
import scipy.odr.odrpack as odrpack
#from uncertainties import ufloat
#from uncertainties.umath import *
#from uncertainties import unumpy

import sympy as sp
sf = sp.sympify

import scipy.constants as cm
import pint as pn
ureg = pn.UnitRegistry(system='mks')
ureg.default_format = '~P'
pe = ureg.parse_expression

## Konstanten und Einheiten
CM = pd.DataFrame(cm.physical_constants).T
CM['c'] = CM.index
CM = CM.reset_index(drop=True)
def getpc(quant):
    DF = CM
    DF = DF[DF['c']==quant].reset_index(drop=True)
    quan = DF[0][0] * ureg(DF[1][0])
    return quan
def addnoise(X,scale=0.01,lin=False):
    if not lin:
        mag = scale*(np.max(X.m)-np.min(X.m))
        return np.random.normal(0,mag,len(X.m))*X.u
    if lin:
        return np.random.normal(0,1,len(X.m))*X*scale
def uODR(func,beta0,xdata,ydata,**kwargs):
    xfit = kwargs.get('xfit',xdata)
    sx = kwargs.get('sx',None)
    sy = kwargs.get('sy',None)
    model = Model(func)
    data = RealData(xdata,ydata, sx=sx, sy=sy)
    odr = ODR(data, model, beta0=beta0)
    out = odr.run()
    # Estimated parameter values, of shape q
    p = out.beta
    # Standard errors of the estimated parameters, of shape p
    perr = out.sd_beta
    up =unumpy.uarray(p, perr)
    yfit=func(p, xfit) 
    return up , p, yfit, out

In [2]:
g = getpc('standard acceleration of gravity')
m_e = getpc('electron mass')
e = getpc('elementary charge')
c = getpc('speed of light in vacuum')
hp = getpc('Planck constant')
e0 = getpc('electric constant')
G = getpc('Newtonian constant of gravitation')
lamC=hp/c/m_e

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Quellcode an/aus schalten."></form>''')

# Bragg-Reflexion


### Aufgabe 1
https://www.leifiphysik.de/optik/beugung-und-interferenz/aufgabe/roentgen-strahlung-und-bragg-abitur-1994-lk-a4-3
### Aufgabe 2
https://www.leifiphysik.de/optik/beugung-und-interferenz/aufgabe/beschriften-eines-graphens
### Aufgabe 3
https://www.leifiphysik.de/optik/beugung-und-interferenz/aufgabe/netzebenenabstand-von-lif-kristallen
### Aufgabe 4
https://www.leifiphysik.de/optik/beugung-und-interferenz/aufgabe/wellenlaengenmessung-mit-einem-lif-kristall


Eine weitere mögliche Aufgabe:

https://www.leifiphysik.de/optik/beugung-und-interferenz/aufgabe/bragg-reflexion-mit-mikrowellen

### Netzebenenabstände der hexagonalen Struktur von Graphit

In [4]:
def dhkl(h,k,l,a,c): return a/np.sqrt(4/3*(h**2+k**2+h*k)+a**2/c**2*l**2)
dhkl(3,1,1,pe('2.47E-10m'),pe('6.79E-10m'))

lim=5
for h in range(lim):
    for k in range(lim):
        for l in range(lim):
            try: 
                d = dhkl(h,k,l,pe('2.47E-10m'),pe('6.79E-10m'))
                if (d.m>=1E-11) & ~((h == 0) & (k == 0) & (l == 0)): #& d.m<=3.2E-10:
                    print(d)
                    print("(h,k,l)=(%d,%d,%d)"%(h,k,l))
                    
            except: pass

6.789999999999999e-10 m
(h,k,l)=(0,0,1)
3.3949999999999996e-10 m
(h,k,l)=(0,0,2)
2.2633333333333333e-10 m
(h,k,l)=(0,0,3)
1.6974999999999998e-10 m
(h,k,l)=(0,0,4)
2.1390827473475632e-10 m
(h,k,l)=(0,1,0)
2.040234185484564e-10 m
(h,k,l)=(0,1,1)
1.8098038100865708e-10 m
(h,k,l)=(0,1,2)
1.5546302564171882e-10 m
(h,k,l)=(0,1,3)
1.3296888237063246e-10 m
(h,k,l)=(0,1,4)
1.0695413736737816e-10 m
(h,k,l)=(0,2,0)
1.0565147450699625e-10 m
(h,k,l)=(0,2,1)
1.020117092742282e-10 m
(h,k,l)=(0,2,2)
9.670083896517608e-11 m
(h,k,l)=(0,2,3)
9.049019050432854e-11 m
(h,k,l)=(0,2,4)
7.130275824491877e-11 m
(h,k,l)=(0,3,0)
7.09128381477691e-11 m
(h,k,l)=(0,3,1)
6.978037293102346e-11 m
(h,k,l)=(0,3,2)
6.800780618281879e-11 m
(h,k,l)=(0,3,3)
6.573879445381355e-11 m
(h,k,l)=(0,3,4)
5.347706868368908e-11 m
(h,k,l)=(0,4,0)
5.331197953036687e-11 m
(h,k,l)=(0,4,1)
5.282573725349813e-11 m
(h,k,l)=(0,4,2)
5.2044084597958765e-11 m
(h,k,l)=(0,4,3)
5.10058546371141e-11 m
(h,k,l)=(0,4,4)
2.1390827473475632e-10 m
(h,k,l)

/home/lukas/.local/lib/python3.7/site-packages/pint/quantity.py:888: RuntimeWarning: divide by zero encountered in double_scalars
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)
